In [ ]:
! pip install -r requirements.txt --quiet

# Connecting to a Remote MCP Server with Semantic Kernel

This notebook demonstrates how to connect to a remote MCP Server using **Semantic Kernel's** `MCPStreamableHttpPlugin`. The **Model Context Protocol (MCP)** enables scalable and modular tool integration across distributed systems. 

<br/>

> **Why Use Model Context Protocol (MCP)?**
>
>MCP allows agents to discover, invoke, and manage tools dynamically across remote servers.  
>It promotes modularity, scalability, and separation of concerns, making it easier to maintain and extend AI systems as they grow in complexity.

In [29]:
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent,ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents.chat_history import ChatHistory  
from dotenv import load_dotenv
from os import environ
import asyncio

from semantic_kernel.connectors.mcp import MCPStreamableHttpPlugin

load_dotenv(override=True)

kernel = Kernel()

kernel.add_service(AzureChatCompletion(
    service_id="chat",
    deployment_name=environ["AZURE_OPENAI_MODEL"],
    base_url=environ["AZURE_OPENAI_ENDPOINT"],
    api_key=environ["AZURE_OPENAI_API_KEY"] ))





In [30]:
instructions = """You are AutoSales Analyst, an AI agent specialized in analyzing automotive sales data.

Your objectives:
- Dynamically choose and call the appropriate tool(s) to answer user questions about sales, customers, or products.
- Always include customer names alongside IDs in any output.
- Always display monetary amounts in $USD (e.g., $123.45).
- Use filters and aggregations as needed to generate insights from sales orders, products, and customers.
- Compute totals, revenue, discounts, and other metrics from nested order data.
- Return structured, concise results suitable for analysis or reporting.
- Avoid hardcoding analytics; rely on the MCP tools and their parameters.
- Clarify ambiguous queries before performing analysis.
- Treat the tools as the source of truth; do not expose raw database internals.
- Return customer_id and product_id alongside names in all outputs.

Behavioral guidance:
- For customer-related questions, resolve names and IDs before analyzing orders.
- For product-related questions, resolve categories or IDs before analyzing sales.
"""

In [31]:

sales_plugin = MCPStreamableHttpPlugin(
    name="sales",
    url=f"{environ['MCP_SERVER_URL']}",
)

await sales_plugin.connect()

agent = ChatCompletionAgent(
    kernel=kernel, 
    name="SalesAgent", 
    instructions=instructions,
    plugins=[sales_plugin, ]
)


In [32]:
messages = [
    "Which products generated the most revenue?",
    "Drill down into customer details product with the highest revenue.",
    "Which customer had the highest sales for this product?",
]


In [33]:

response = await agent.get_response(messages=messages[0])
print(response)


To identify the products generating the most revenue, I'll calculate the total revenue for each product by aggregating the `line_unit_price` values across all relevant sales orders. Please allow me a moment to process the data.
Here are the products ranked by total revenue generation:

1. **Brake Pad** (Product ID: 0)  
   - Total Revenue: $12,361.50

2. **Oil Filter** (Product ID: 1)  
   - Total Revenue: $4,334.73

3. **Spark Plug** (Product ID: 2)  
   - Total Revenue: $1,983.25

The results show that Brake Pads contribute the highest revenue, followed by Oil Filters and Spark Plugs. Let me know if you'd like a deeper dive into specific customers or regions.


In [34]:
response2 = await agent.get_response(messages=messages[1], thread=response.thread)
print(response2)



The **Brake Pad** (Product ID: 0), the product with the highest revenue, has been purchased by multiple customers, contributing to its significant revenue generation. Below are the key customer details and their contributions to total revenue from Brake Pads:

### Top Contributing Customers:
1. **Customer:** AutoZone (Customer ID: 2)  
   - **Industry:** Distributor  
   - **Region:** NA  
   - **Account Manager:** Carol Lee  
   - **Brake Pad Revenue:** $2,936.00

2. **Customer:** Bosch (Customer ID: 3)  
   - **Industry:** OEM  
   - **Region:** EU  
   - **Account Manager:** David Wong  
   - **Brake Pad Revenue:** $3,134.00

3. **Customer:** NAPA (Customer ID: 4)  
   - **Industry:** Distributor  
   - **Region:** NA/EU  
   - **Account Manager:** Ellen Garcia  
   - **Brake Pad Revenue:** $3,014.50

4. **Customer:** GM (Customer ID: 1)  
   - **Industry:** OEM  
   - **Region:** NA/EU  
   - **Account Manager:** Bob Smith  
   - **Brake Pad Revenue:** $2,545.50

5. **Customer:** F

In [35]:
response3 = await agent.get_response(messages=messages[2], thread=response.thread)
print(response3)

The customer with the highest sales for the **Brake Pad** (Product ID: 0) is:

### Customer: **Bosch**  
- **Customer ID:** 3  
- **Industry:** OEM  
- **Region:** EU  
- **Account Manager:** David Wong  
- **Total Brake Pad Revenue:** $3,134.00  

Bosch has contributed the most revenue from the sales of Brake Pads, indicating strong demand in the OEM sector and a strong presence in the EU market.
